构造双半月分类数据集，并使用KNN和MLP进行分类

In [5]:
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons
from plotly import express as px
import os

os.environ["CUDA_VISIBLE_DEVICES"]="0"
X, y = make_moons(n_samples=1000, noise=0.2)
moons = pd.DataFrame(pd.concat([pd.DataFrame(X,columns=['x1', 'x2']), pd.DataFrame(y,columns=["label"])], axis=1))

fig = px.scatter(moons,
                 x='x1',
                 y='x2',
                 color='label',
                 color_continuous_scale=['blue', 'red'],
                 title='散点图 - 不同标签用不同颜色表示')

fig.show()



## 1、使用KNN进行分类

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

KNN = KNeighborsClassifier(n_neighbors=10)
KNN.fit(X,y)  # predict_centre为使用K_Means计算出来的中心，在这里相当于数据标签
# a = pd.DataFrame([5,5]).T

result = KNN.predict(X) # 转置之后才是 1行2列 的数据，或者使用KNN.predict(pd.array([5,5]))

import plotly.graph_objects as go

# 定义网格边界（稍微扩展数据范围以更好地显示边界）
x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1

# 创建网格点
xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.02),
                     np.arange(y_min, y_max, 0.02))

# 预测网格点的类别
Z = KNN.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)

fig = go.Figure()

# 添加决策边界背景（使用等高线图）
fig.add_trace(go.Contour(
    x=np.arange(x_min, x_max, 0.02),
    y=np.arange(y_min, y_max, 0.02),
    z=Z,
    showscale=False,
    opacity=0.4,
    colorscale='Viridis'  # 可根据类别数调整颜色映射
))

# 添加原始数据点
fig.add_trace(go.Scatter(
    x=X[:, 0],
    y=X[:, 1],
    mode='markers',
    marker=dict(
        color=y,
        colorscale='Viridis',
        showscale=False
    ),
    showlegend=False
))

# 更新布局
fig.update_layout(
    title='KNN Decision Boundary',
    xaxis_title='Feature 1',
    yaxis_title='Feature 2'
)

# 显示图形
fig.show()

#
# print(result_cluster)





## 2、使用MLP进行分类

In [8]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=42)

mlp = Sequential()
mlp.add(Dense(units=20,input_dim=2,activation="sigmoid"))
mlp.add(Dense(units=1,activation="sigmoid"))
mlp.summary()

mlp.compile(optimizer="adam",loss="binary_crossentropy",)

# os.environ["CUDA_VISIBLE_DEVICES"]="0"
with tf.device('/GPU:0'):
    mlp.fit(X_train,y_train,epochs=5000, verbose=0)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 20)                60        
                                                                 
 dense_5 (Dense)             (None, 1)                 21        
                                                                 
Total params: 81
Trainable params: 81
Non-trainable params: 0
_________________________________________________________________
